<a href="https://colab.research.google.com/github/alu0100880755/Logistica/blob/master/TSP_4%20(Precedencias).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Instalamos lo necesario
!pip install ortools
from ortools.linear_solver import pywraplp

#Importamos las librerías necesarias
import random
import math
import matplotlib.pyplot as plt

#Declaramos las variables
n = 15
todos = range(n)
otros = range(1,n)

#Semilla para que empiece por un punto similar siempre y poder comparar con varios modelos
random.seed(8888)

points = [(random.randint(0,100),random.randint(0,100)) for i in range(n)]
dist= { (i,j) : random.randint(1,100) for i in todos for j in todos if i!=j }


#Precisión
EPS = 0.001


#####################################################

    #Muestreo de grafos

#Dibujar una ruta dada como una lista de arcos
def dibuja(selected):                
    plt.plot([p[0] for p in points], [p[1] for p in points], 'bo')
    for (i,j) in selected:
        plt.plot([points[i][0],points[j][0]], [points[i][1],points[j][1]], 'g-')

######################################################

def TSP_4():
    solver = pywraplp.Solver('TSP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    #VARIABLES
    
    #Arcos elegidos
    x = {}
    
    for i in todos:
        for j in todos:
            if i!=j:
                x[i,j] = solver.BoolVar('x[%i,%i]' % (i,j))
                
    #Posición de i (será 1 si i antes que j)
    
    v = {}
        
    for i in todos:
        for j in todos:
            if i!=j:
                v[i,j] = solver.BoolVar('v[%i,%i]' % (i,j))

    #Función objetivo
    solver.Minimize(solver.Sum( dist[i,j]*x[i,j] for i in todos for j in todos if i!=j ))
    
    #Restricciones
    [ solver.Add(  solver.Sum(x[i,j] for j in todos if i!=j) == 1  )  for i in todos ]
    [ solver.Add(  solver.Sum(x[j,i] for j in todos if i!=j) == 1  )  for i in todos ]
    [ solver.Add(  v[i,j] + v[j,i] == 1 ) for i in todos for j in todos if i!=j ]
    [ solver.Add(  v[i,j] + v[j,k] <= v[i,k] + 1 ) for i in todos for j in todos for k in todos if j!=i and i!=k and j!=k]
    [ solver.Add(  x[i,j] >= x[i,j] ) for i in todos for j in todos if j!=i ]    
    
    
    solver.Solve()

    print('\nDistancia total de la ruta = ', solver.Objective().Value(),'kilometros')
    print('\nRuta de distancia mínima:\n')
    for i in todos:
        for j in todos: 
            if i!=j and x[i,j].solution_value() > EPS :
                print('\tDe %d a %d con distancia = %d Km'  % (i,j,dist[i,j]))
                dibuja({(i,j): dist[i,j]}) #Mostrar la ruta
    
    
    #Muestreo de datos
    
    print("\n\tDATOS DE INTERES")

    milisegundos = solver.WallTime()
    segundos = round((milisegundos/1000),2)
    minutos = round((segundos/60),2)
    print("\nTiempo Empleado: ", milisegundos, " milisegundos, | ", segundos , " segundos | ", minutos, " minutos ")
    print("Iteraciones: ", solver.Iterations())
    print("Nodos: ", solver.nodes())

    
TSP_4()
